In [1]:
import sys
print("Python Version:", sys.version)

Python Version: 3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
import re
import os

## Import Data

In [6]:
import csv

# get path to processed data
data_dir = os.path.abspath('../data/processed')

with open(f'{data_dir}/sensors.csv', 'r', newline='') as csvfile:
    csv_sens = csv.reader(csvfile, delimiter=',')
    for row in csv_sens:
        sensors = row

features_df = pd.read_csv(f'{data_dir}/features.csv', index_col=0)
targets_df = pd.read_csv(f'{data_dir}/targets.csv', index_col=0)

## Basic Models

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.svm import SVC, LinearSVC

In [8]:
# number of important features
n_feats = 150

# results
results = {}

# Features
X = features_df

# loop through y's and get feature importances
for col in targets_df.columns.values:

    # single test result
    result = {}

    # set y's
    y = targets_df.loc[:,col]

    # train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.25, random_state=42, stratify=y)

    # initialize model
    model = RandomForestClassifier(n_estimators=100, random_state=42)

    # fit model
    model.fit(X_train, y_train)
    y_test_pred = model.predict(X_test)
    y_test_pred = pd.Series(y_test_pred)
    y_test_pred.index = y_test.index
    y_test_pred.name = 'pred_'+col

    # store the predicted/actuals
    result['actual'] = deepcopy(y_test)
    result['predicted'] = deepcopy(y_test_pred)

    # score
    result['recall'] = recall_score(y_test, y_test_pred, average='weighted')
    result['precision'] = precision_score(y_test, y_test_pred, average='weighted')

    try:
        # feature importances
        feat_imp = model.feature_importances_
        feats = X_train.columns
        indices = np.argsort(feat_imp)[::-1]
        indices = indices[0:n_feats]

        # get the features
        result['features'] = deepcopy(feats[indices])

        # get the features summary (in order of importance)
        # split sensor name from time
        feats_df = pd.Series(feats[indices]).str.split(' ', expand=True)
        feats_df.columns = ['sensor','time']
        feats_df['time'] = feats_df['time'].astype(float)

        # summarize the times for each sensor
        feats_df = feats_df.groupby('sensor',sort=False).agg(['mean','min','max','std','count'])

        # use a multi-index so all DFs can be joined into master sensor DF
        m_ind = pd.MultiIndex.from_product([[col],feats_df.index.values])
        feats_df.index = m_ind

        # put feature summary in results
        result['sensors'] = deepcopy(feats_df)
    except AttributeError as attrib:
        print(attrib)
    
    # put result into results
    results[col] = deepcopy(result)

    #progress tracking
    print(col)

cooler_eff
valve_perc
pump_leak
accu_prs
fault_id


### Most Important Sensors and Times

In [9]:
# reclaim some ram
del X_train, y_train, X_test, y_test#, model

# put results into dataframe
master_sensor_df = pd.DataFrame()

for k in results.keys():
    if master_sensor_df.empty:
        master_sensor_df = results[k]['sensors']
    else:
        master_sensor_df = master_sensor_df.append(results[k]['sensors'])

In [10]:
master_sensor_df

time                               
                      mean    min    max        std count
cooler_eff PS6   31.557843   2.36  59.22  18.307851    51
           PS2   22.365714  11.65  54.89  13.890664    14
           PS5   31.166458   0.35  59.22  17.759973    48
           EPS1  16.053333   1.55  59.16  19.835953    18
           PS4   38.856667  35.66  42.59   3.496031     3
           PS1   21.327500  12.47  54.47  12.538076    12
           TS4   33.500000  24.00  43.00  13.435029     2
           FS2   29.200000  29.20  29.20        NaN     1
           CP     8.000000   8.00   8.00        NaN     1
valve_perc PS1    9.723261   9.45  10.19   0.176409    46
           PS3    9.743922   9.40  10.21   0.204764    51
           PS2    9.582632   9.34   9.95   0.165573    38
           FS1    9.866667   9.60  10.20   0.216025     6
           EPS1   9.966250   9.86  10.10   0.075012     8
           SE    11.000000  11.00  11.00        NaN     1
pump_leak  SE    35.270270   1.00  60.00  15.379728    37
           FS1   40.267059  11.10  59.90  15.110485    85
           EPS1   7.463214   1.29  49.79  11.368839    28
accu_prs   PS1    7.568269   0.06  50.32  14.139612    52
           PS3   32.771176   0.05  49.41  20.909947    17
           FS1   39.360000  19.20  49.50  12.349615     5
           TS3   35.714286  17.00  55.00  13.561149     7
           TS2   29.642857   4.00  56.00  18.002594    14
           TS1   29.357143   1.00  60.00  17.260926    14
           TS4   32.285714   4.00  60.00  19.722804    14
           PS2   28.576923  19.05  50.04  14.566720    13
           FS2   29.885714   5.60  52.50  15.708491    14
fault_id   PS2    9.986279   9.51  10.40   0.282430    43
           PS1    9.524559   0.42  10.75   2.289480    68
           FS1   10.287500   9.80  10.80   0.331393     8
           TS1   25.000000  15.00  37.00  11.135529     3
           PS3    9.776667   9.51  10.13   0.186611    15
           SE    25.500000  20.00  31.00   7.778175     2
           TS3   13.000000  13.00  13.00        NaN     1
           EPS1  10.370000   9.88  10.84   0.281736     9
           TS2   57.000000  57.00  57.00        NaN     1

In [11]:
# empty dataframe with 3 columns
master_score_df = pd.DataFrame(columns=['fault','precision','recall'])

# loop through all the rows
for k in results.keys():
    # value for each column
    fault = k
    prec = results[k]['precision']
    rec = results[k]['recall']
    
    # build dictionary for the row with keys=column names
    score = {'fault':k, 'precision':prec, 'recall':rec}
    
    # add the row with ignore_index =True
    master_score_df = master_score_df.append(score, ignore_index=True)

In [12]:
master_score_df

,fault,precision,recall
0,cooler_eff,1.000000,1.000000
1,valve_perc,1.000000,1.000000
2,pump_leak,0.994581,0.994490
3,accu_prs,0.994610,0.994490
4,fault_id,0.994490,0.991736
